In [2]:
from glob import glob
import json
import random
import os
from malaysian_sft import accept

In [3]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions'
]

In [4]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

In [8]:
files = glob('/home/husein/ssd2/gov.my/ultrachat-*.jsonl')
files = [f for f in files if 'coder' not in f and 'textbooks' not in f]
lines = 0
for f in files:
    with open(f) as fopen:
        for l in fopen:
            lines += 1
lines

186496

In [10]:
files

['/home/husein/ssd2/gov.my/ultrachat-crossref-melayu.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-gov.my.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-astroawani-malay.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-lom-agc.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-epenerbitan-malay.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-jurnaldbp-malay.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-jurnaldbp.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-ms-wikipedia.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-crossref-melayu-malay.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-maktabahalbakri.com.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-muftiwp.gov.my.texts.jsonl',
 '/home/husein/ssd2/gov.my/ultrachat-hansard-malay.jsonl']

In [11]:
from tqdm import tqdm

data_instructions = []

count, count1 = 0, 0
for file in files:
    with open(file) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
#             if l[1]['content'] is None:
#                 print(l)
#                 continue
            
            if 'glaive_coder_raw_text' in file:
                code_instruct = True
            else:
                code_instruct = False
                
            context = l[0]['content']
            
            if not code_instruct:
                l = l[1:]
                
            inputs = []
            for no, r in enumerate(l):
                
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content_ms') or l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    current_text = r.get('content_ms') or r.get('content') or ''
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content_ms') or l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if not code_instruct and r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if not code_instruct and r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break

                role = roles[r['role']]
                
                if no == 0 and not code_instruct and ('crossref-melayu' in file or random.random() > 0.7):
                    s = f"{role}: {context}\n\n{current_text}"
                    count += 1
                else:
                    s = f"{role}: {current_text}"
                    
                inputs.append(s)
                count1 += 1

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue

            data = '\n'.join(inputs).strip()
            
            if not len(data):
                continue
                
            a = {
                'prompt_input': None,
                'input': data,
                'output': None
            }
            data_instructions.append(a)
        
        
len(data_instructions)

1296it [00:00, 1416.35it/s]
10128it [00:04, 2355.05it/s]
60198it [00:13, 4386.85it/s]
8044it [00:03, 2071.62it/s]
4567it [00:01, 2362.75it/s]
6440it [00:02, 2311.50it/s]
1734it [00:01, 1257.62it/s]
4408it [00:01, 2639.26it/s]
9959it [00:04, 2250.23it/s]
3350it [00:01, 2139.34it/s]
3834it [00:01, 2115.64it/s]
72538it [00:22, 3239.42it/s]


185552

In [17]:
def generate_and_tokenize_prompt(row, validate = True):
    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except BaseException:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human)
            outputs.append(bot)
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    chat = []
    if 'function_call' in row:
        prompt = f"""
Below is functions provided to convert user input into function parameters,
<FUNCTION>
{row['function_call']}
</FUNCTION>
Sometime user input is not able to convert to function parameters, so please ignore it and reply as normal chatbot.
""".strip()
        chat.append({'role': 'system', 'content': prompt})
    for input, output in zip(inputs, outputs):
        chat.extend([
            {'role': 'user', 'content': input.strip()},
            {'role': 'assistant', 'content': output.strip()},
        ])
    if validate:
        for c in chat:
            if c['role'] == 'assistant' and not accept(c['content']):
                return
    return chat

In [18]:
generate_and_tokenize_prompt(data_instructions[0])

[{'role': 'user',
  'content': 'English \n\n\n\nconcepts of patriarchal families were yet in sync; and individual rights and notions of justice for \n\n\n\nboth, being at polar ends. Points of weighty contention, while outside of the purview of this \n\n\n\narticle, nevertheless contributed to the outcome of the MTO.      \n\n\n\n\n\n\n\nConclusion \n\n\n\nIn the 30-year period that the Mui Tsai phenomenon held sway in British Malaya the Colonial \n\n\n\ngovernment‟s agenda to answer the foundational question “how best to save or protect these \n\n\n\nyoung girls”\n162\n\n\n\n remained unsuccessfully resolved. This article explored the Mui Tsai practice \n\n\n\nwithin British Malaya, specifically the Strait Settlements and unpackaged some of the general \n\n\n\nunderlying complexities experienced transnationally as well as locally. It demonstrated how the \n\n\n\nunderstanding of fundamental legal and non-legal notions surrounding this question came into \n\n\n\nconflict in the Strait 

In [20]:
post = []
for d in tqdm(data_instructions):
    l = json.dumps(d)
    if 'terjemah' in l.lower():
        continue
    if 'artikel itu' in l.lower():
        continue
    r = generate_and_tokenize_prompt(d)
    if r is not None:
        post.append(d)

100%|██████████████████████████████████| 185552/185552 [05:01<00:00, 615.08it/s]


In [21]:
import pandas as pd

df = pd.DataFrame(post)
df.head()

,prompt_input,input,output
0,None,<manusia>: English \n\n\n\nconcepts of patria...,None
1,None,<manusia>: © 2022 Sains Insani \n\n\n\n\n\n\n...,None
2,None,<manusia>: a ini. Sumbang saran dilakukan supa...,None
3,None,<manusia>: 10: 3-4 (2018) 27–33 | www.sainshum...,None
4,None,<manusia>: they lost their freedom and rights...,None


In [22]:
df.to_parquet('prepared-malaysian-ultrachat.parquet')

In [30]:
df.describe()

,prompt_input,input,output
count,0,149054,0
unique,0,149054,0
top,NaN,<manusia>: English \n\n\n\nconcepts of patria...,NaN
freq,NaN,1,NaN


In [26]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-malaysian-ultrachat.parquet",
    path_in_repo='data/malaysian_ultrachat-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-malaysian-ultrachat.parquet:   0%|          | 0.00/318M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/2785513f27271b7692acefc212a4e62ffb426576', commit_message='Upload data/malaysian_ultrachat-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='2785513f27271b7692acefc212a4e62ffb426576', pr_url=None, pr_revision=None, pr_num=None)